In [88]:
import xmltodict
import json
import os
import requests

In [89]:
filename = 'F600-2.7894cbdc.json'
with open(filename, 'r') as f:
  text = f.read()

In [90]:
j = json.loads(text)
with open('性能数据.json', 'r') as f:
  file_template = json.load(f)

In [91]:
lang_en_zh = {
  'number-of-test-sample': '测试样品编号',
  'volume-ratio-of-reinforcement-in-test-sample': '测试样品含量',
  'test-date': '测试日期',
  'tester': '测试人',
  'youngs-modulus': '弹性模量',
  'specifies-the-plastic-elongation-strength': '规定塑性延伸强度',
  'tensile-strength': '抗拉强度',
  'break-strength': '断裂强度',
  'percentage-total-elongation-at-fracture': '断裂总伸长率',
  'percentage-elongation-after-fracture': '断后伸长率',
  'data-of-tensile-testing': '拉伸数据',
  'tensile-curve': '拉伸曲线', 
  'elastic-modulus-of-vibration-method': '振动法弹性模量',
  'fracture-morphology': '断口形貌',
  'impact-toughness': '冲击韧性',
  'impact-fracture-morphology': '冲击断口形貌',
  'microyield-strength': '微屈服强度',
  'rp': '规定非比例延伸强度',
  'strength': '强度',
  'percentage-elongation-at-maximum-force': '最大力总伸长率',
  
}

In [92]:
# 力学特性
mechanical_property = j['SJTU-al-mmc-property']['mechanical-property']

number_of_test_sample = mechanical_property.get('number-of-test-sample')
volume_ratio_of_reinforcement_in_test_sample = mechanical_property.get('volume-ratio-of-reinforcement-in-test-sample')['value']
test_date = mechanical_property.get('test-date')
tester = mechanical_property.get('tester')
youngs_modulus = mechanical_property.get('youngs-modulus', {}).get('value')
if mechanical_property.get('specifies-the-plastic-elongation-strength'):
  record = mechanical_property['specifies-the-plastic-elongation-strength']['specifies-the-plastic-elongation-strength-record']
  rp = record.get('rp')
  strength = record.get('strength', {}).get('value')
tensile_strength = mechanical_property.get('tensile-strength', {}).get('value')
break_strength = mechanical_property.get('break-strength', {}).get('value')
percentage_elongation_at_maximum_force = mechanical_property.get('percentage-elongation-at-maximum-force', {}).get('value')
tensile_curve = mechanical_property.get('tensile-curve')['file']['name']

download_url = 'http://matdata.shu.edu.cn' + mechanical_property.get('tensile-curve')['file']['url']


In [93]:
# 获取文件名
_, file = os.path.split(filename)
# 创建 output/xxxx/文件夹
dir_name = os.path.join('output/', file[0:-5])
# 存放二进制的文件夹
files_dir = os.path.join(dir_name, 'files')
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

if not os.path.exists(files_dir):
    os.makedirs(files_dir)

In [94]:
data_template = {
            "meta": {
                "数据 ID": "1",
                "标题": "铝基复合材料 性能数据1",
                "DOI": "",
                "数据摘要": "铝基复合材料 性能数据1",
                "关键词": "铝基复合材料 性能数据",
                "来源": "MGE-SOURCE_HEADER v1 0100 10 #",
                "引用": "",
                "其他信息": "project: 2018YFB0704400；subject: 2018YFB0704401",
                "数据生产机构": "SJTU",
                "数据生产者": "SJTU",
                "公开时间": "0",
                "公开范围": "public"
            },
            "content": {
                "力学性能": {
                    "测试样品编号": number_of_test_sample,
                    "测试样品含量": volume_ratio_of_reinforcement_in_test_sample,
                    "测试日期": test_date,
                    "测试人": tester,
                    "弹性模量": youngs_modulus,
                    "规定塑性延伸强度": [
                        {
                            "规定非比例延伸强度": rp,
                            "强度": strength,
                        },
                    ],
                    "抗拉强度": tensile_strength,
                    "断裂强度": break_strength,
                    "最大力总伸长率": percentage_elongation_at_maximum_force,
                    "拉伸曲线": [
                        os.path.join('files/', tensile_curve),
                    ]
                }
            }
        }


file_template['data'].append(data_template)

In [95]:
# 下载文件
r = requests.get(download_url).content
# 文件必须单独放一个文件夹里
with open(os.path.join(files_dir, tensile_curve), 'wb+') as f:
  f.write(r)

out_file = os.path.join(dir_name, file)

with open(out_file, 'w+') as f:
  json.dump(file_template, f, ensure_ascii=False)


In [96]:
file_template

{'template': {'_id': 3438,
  '力学性能': {'_type': '容器型',
   'r': False,
   '测试样品编号': {'_type': '字符串型', 'r': False},
   '测试样品含量': {'_type': '字符串型', 'r': False},
   '测试日期': {'_type': '字符串型', 'r': False},
   '测试人': {'_type': '字符串型', 'r': False},
   '弹性模量': {'_type': '数值型', 'r': False, '_unit': 'Gpa'},
   '规定塑性延伸强度': {'_type': '表格型',
    'r': False,
    '规定非比例延伸强度': {'_type': '数值型', 'r': False, '_unit': 'Mpa'},
    '强度': {'_type': '数值型', 'r': False}},
   '抗拉强度': {'_type': '数值型', 'r': False, '_unit': 'Mpa'},
   '断裂强度': {'_type': '数值型', 'r': False, '_unit': 'Mpa'},
   '最大力总伸长率': {'_type': '数值型', 'r': False, '_unit': '%'},
   '拉伸曲线': {'_type': '文件', 'r': False}}},
 'data': [{'meta': {'数据 ID': '1',
    '标题': '铝基复合材料 性能数据1',
    'DOI': '',
    '数据摘要': '铝基复合材料 性能数据1',
    '关键词': '铝基复合材料 性能数据',
    '来源': 'MGE-SOURCE_HEADER v1 0100 10 #',
    '引用': '',
    '其他信息': 'project: 2018YFB0704400；subject: 2018YFB0704401',
    '数据生产机构': 'SJTU',
    '数据生产者': 'SJTU',
    '公开时间': '0',
    '公开范围': 'public'},
   '